In [24]:
# Cell 1: Work Done - Summary of Approaches Tried
"""
1. Different feature extraction techniques were tested:
   - Raw pixel values
   - Histogram of Oriented Gradients (HOG) with Histogram Equalization ✅ (Best)
   - Local Binary Patterns (LBP)
   - PCA for dimensionality reduction
   - Canny Edge Detection + HOG

2. Among these, HOG with histogram equalization provided the best performance.

3. The final workflow includes:
   - Loading and preprocessing images
   - Extracting features using HOG
   - Standardizing the feature set
   - Training SVM and MLP classifiers
   - Evaluating accuracy and performance
"""


'\n1. Different feature extraction techniques were tested:\n   - Raw pixel values\n   - Histogram of Oriented Gradients (HOG) with Histogram Equalization ✅ (Best)\n   - Local Binary Patterns (LBP)\n   - PCA for dimensionality reduction\n   - Canny Edge Detection + HOG\n\n2. Among these, HOG with histogram equalization provided the best performance.\n\n3. The final workflow includes:\n   - Loading and preprocessing images\n   - Extracting features using HOG\n   - Standardizing the feature set\n   - Training SVM and MLP classifiers\n   - Evaluating accuracy and performance\n'

In [25]:
# Cell 2: Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

In [26]:
# Cell 3: Load images and labels
def load_images_from_folder(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize for consistency
            images.append(img)
            labels.append(label)
    return images, labels

mask_images, mask_labels = load_images_from_folder("FaceMaskDetection_dataset/with_mask", label=1)
no_mask_images, no_mask_labels = load_images_from_folder("FaceMaskDetection_dataset/without_mask", label=0)

# Combine data
X = np.array(mask_images + no_mask_images)
y = np.array(mask_labels + no_mask_labels)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [27]:
# 1. Histogram of Oriented Gradients (HOG) 
def extract_hog_features(images):
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Histogram Equalization for Improved Contrast
        gray = cv2.equalizeHist(gray)

        # Optimized HOG Parameters
        hog_features = hog(
            gray,
            orientations=9,                  # Balanced gradient directions
            pixels_per_cell=(8, 8),          # Optimal for face detection
            cells_per_block=(2, 2),          # Stronger feature stability
            block_norm='L2-Hys',            
            feature_vector=True
        )
        features.append(hog_features)
    return np.array(features)

# 2. Local Binary Patterns (LBP)
def extract_lbp_features(images, P=8, R=1):
    features = []
    for img in images:
        lbp = local_binary_pattern(img, P, R, method="uniform")
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype("float")
        hist /= hist.sum()  # Normalize histogram
        features.append(hist)
    return np.array(features)

# 3. PCA for Dimensionality Reduction
def apply_pca(features, n_components=50):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(features)

# 4. Canny Edge Detection + HOG
def extract_canny_hog_features(images):
    features = []
    for img in images:
        edges = cv2.Canny(img, 100, 200)  # Apply Canny edge detection
        hog_feature = hog(edges, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9, visualize=False)
        features.append(hog_feature)
    return np.array(features)



# Extract features using HOG
X_hog = extract_hog_features(X)

In [28]:
# Cell 5: Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42, stratify=y)

In [29]:
# Cell 6: Train and Evaluate Models
def train_and_evaluate(X_train, X_test, y_train, y_test):
    # Standardizing Features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # SVM Classifier
    svm_model = SVC(kernel='rbf', C=10, gamma='auto')
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)

    # MLP Classifier
    mlp_model = MLPClassifier(hidden_layer_sizes=(256, 128, 64), 
                              activation='relu', 
                              max_iter=1000, 
                              random_state=42)
    mlp_model.fit(X_train, y_train)
    y_pred_mlp = mlp_model.predict(X_test)

    # Evaluation
    print("=== SVM Classifier ===")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")
    print(classification_report(y_test, y_pred_svm))

    print("=== MLP Classifier ===")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_mlp):.2f}")
    print(classification_report(y_test, y_pred_mlp))
    
    return svm_model, mlp_model, scaler, y_pred_svm, y_pred_mlp

# Train and evaluate models
svm_model, mlp_model, scaler, y_pred_svm, y_pred_mlp = train_and_evaluate(X_train, X_test, y_train, y_test)


=== SVM Classifier ===
Accuracy: 0.94
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       386
           1       0.93      0.95      0.94       433

    accuracy                           0.94       819
   macro avg       0.94      0.93      0.93       819
weighted avg       0.94      0.94      0.94       819

=== MLP Classifier ===
Accuracy: 0.93
              precision    recall  f1-score   support

           0       0.94      0.91      0.92       386
           1       0.92      0.95      0.94       433

    accuracy                           0.93       819
   macro avg       0.93      0.93      0.93       819
weighted avg       0.93      0.93      0.93       819

